<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
# библиотеки для работы и визуализации данных
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# подготовка данных к обучению:
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

# модели машинного обучения:
from sklearn.ensemble import RandomForestClassifier

# метрики для оценки работы моделей:
from sklearn.metrics import (accuracy_score, recall_score, precision_score,
                             f1_score, confusion_matrix,
                             precision_recall_curve, roc_curve, roc_auc_score)


pd.options.mode.chained_assignment = None # скрыть предупреждения

In [2]:
data = pd.read_csv(r'C:\Users\Grine\Downloads\Churn.csv')

In [3]:
data.head(5)
# названия столбцов не соотвествуют стандарту pep-8

In [4]:
# данный код создаёт список, в который помещает все слова с заглавной буквы. Затем соединяет слова используя нижнее подчёркивание;
# после этого приводит все слова к нижнему регистру по каждому названию 
import re
data.columns = [(('_'.join(re.findall('[A-Z][^A-Z]*', name))).lower()) for name in data.columns]

In [5]:
data.info() # необходимо преобразовать признаки. (удалить лишнее и привести к нужному типу данных)

In [6]:
data = data[data['tenure'].notna()] #Избавлясь от 10% значений, содержащих пропуски исходный набора данных не перестаёт давать полную картину об анализируемых данных.
                                    #Однако, мы снижаем риски, что отсутсвие значения, будет неправильно идентифицировано моделью

In [7]:
# преобразуем типы данных
data[data.select_dtypes(include='float64').columns] = (data
                                                       .select_dtypes(include='float64').astype('float32'))
data[data.select_dtypes(include='int64').columns] = (data
                                                       .select_dtypes(include='int64').astype('int32'))

In [8]:
serieses_with_type_object = data.select_dtypes('object') # посмотрим, какие столбцы принимают тип данных object
print(len(set(serieses_with_type_object.iloc[:, 0]))) # множество фамилий. Удалим этот столбец он будет запутывать модель.
print(set(serieses_with_type_object.iloc[:, 1])) # множество стран. Приобразуем с помощью One-hot Encoding
print(set(serieses_with_type_object.iloc[:, 2])) # множество полов. Тут также подойдёт метод OHE.
data = data.drop(columns='surname', axis=1)
data = pd.get_dummies(data) # Дамми-ловушка тут не возникнет. 
                            # Удалять первую колонку не надо. Все 3 страны - разные.
                            # А гендера всего 2.

In [9]:
# Дополнительно из модели стоит удалить такие столбцы, как: row и customer-id. 
# вряд-ли порядковый номер строки и порядковый номер клиента поспособствуют улучшению качества модели
data = data.drop(columns=['row_number', 'customer_id'])

### Исследование задачи

План исследования:

    1) делим данные на выборки;
    2) масштабируем признаки в данных;
    3) подбираем лучшие модели по целевому параметру F1_score (без учёта дисбаланса);

Прорабатываем дисбаланс классов:

    4) Сделали апсемплинг, посмотрели результат (отобрали лучшую модель по F1-мере и ROC-AUC);
    5) Сделали даунсемплинга, отобрали лучшую модель (отобрали лучшую модель по F1-мере и ROC-       AUC);
    6) Отбираем лучшую модель при взвешивание классов (отобрали лучшую модель по F1-мере и ROC-       AUC).

In [12]:
# Создадим 2 группы данных. Целевой признак и все остальные

target = data['exited'] 
features = data.drop(columns=['exited'], axis=1) 

In [13]:
features_train, features_tech, target_train, target_tech = train_test_split(
    features, target, test_size=0.4, random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_tech, target_tech, test_size=0.5, random_state=12345) 


print('Объём выборки для обучения:', "{0:.0%}".format(features_train.shape[0]/features.shape[0]))
print('Объём выборки для валидации:', "{0:.0%}".format(features_valid.shape[0]/features.shape[0]))
print('Объём выборки для теста:', "{0:.0%}".format(features_test.shape[0]/features.shape[0]))

Прибегнем к методу масштабизации признаков. Чтобы избежать утечки признаков. В качестве метода масштабизации будем применять метод стандртизации

In [14]:
numeric = list(data.columns[[0, 1, 2, 3, 4, 7]])
print('Отобранные для стандартизации столбцы:', numeric)

In [15]:
scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

**Данные разбиты на выборки и приведены к станадртному виду. Теперь можно проводить пробное обучение**

### Пробное обучение

In [17]:
%%time

best_model_random = None
best_random_forest_f1_score = 0
best_auc_roc_score_model_random_forest = 0

for d in range(1, 10):
    for n_e in range(10, 200, 10):
        model_random_forest = RandomForestClassifier(max_depth=d,
                                                    n_estimators=n_e,
                                                    random_state=12345)
        model_random_forest.fit(features_train, target_train)
        predictions = model_random_forest.predict(features_valid)
        result = f1_score(target_valid, predictions)
        result_roc = roc_auc_score(target_valid, model_random_forest.predict_proba(features_valid)[:, 1])
        if result > best_random_forest_f1_score:
            best_random_forest_f1_score = result
            best_auc_roc_score_model_random_forest = result_roc
            best_model_random = model_random_forest
        else:
            continue
            
print('Best_f1_sccore_result:', "{:.3f}".format(best_random_forest_f1_score))
print('Best_auc_roc_score_result:', "{:.3f}".format(best_auc_roc_score_model_random_forest))

**По пробному обучению видно, что целевая F1-мера не достигнута необходимо разобраться в чём проблема. Вероятно, связана эта проблема с дисбалансом классов.**

**Учитывая этот фактор, лучше всего показала себя модель линейной регрессии.**

In [19]:
confusion_matrix(target_valid, predictions)

## Борьба с дисбалансом

Для борьбы с дисбалансом будет применяно 3 метода:

    1) апсемплинг;
    2) даунспелинг;
    3) взвешивание классов;
    
Сначала проведём обучение моделей на каждом из методе, а потом посмотрим итоговые результаты.

### Upsempling

C помощью метода SMOTE импортированного из библиотеки imblearn, избавим от дисбаланса класса (метод увеличиваем выборку с меньшим классом).

In [21]:
oversample = SMOTE(random_state=12345)
features_train_up, target_train_up = oversample.fit_resample(features_train, target_train)

In [22]:
method = 'upsempling'                      # метод, который тестируем
best_model_random = None                   # лучшая модель случайного леса
best_random_forest_f1_score = 0            # лучший результат f1-score для случайного леса
best_auc_roc_score_model_random_forest = 0 # ллучший результат auc-roc для случайного леса



for d in range(1, len(features_train.columns)):
    for n_e in range(10, 200, 10):
        model_random_forest = RandomForestClassifier(max_depth=d,
                                                    n_estimators=n_e,
                                                    random_state=12345)
        model_random_forest.fit(features_train_up, target_train_up)
        predictions = model_random_forest.predict(features_valid)
        result = f1_score(target_valid, predictions)
        result_roc = roc_auc_score(target_valid, model_random_forest.predict_proba(features_valid)[:, 1])
        if result > best_random_forest_f1_score:
            best_random_forest_f1_score = result
            best_auc_roc_score_model_random_forest = result_roc
            best_model_random = model_random_forest
        else:
            continue


# создадим списки с результатми 

models = [best_model_random]
f1_results = [best_random_forest_f1_score]
auc_roc_results = [best_auc_roc_score_model_random_forest]
method = (f'{method}/' * len(models)).split('/')[:-1] # технический список. В него будет вносится метод

In [23]:
results1 = pd.DataFrame({
                                    'models': models,
                                    'f1_results': f1_results,
                                    'auc_roc_results': auc_roc_results,
                                    'method': method
                                }).sort_values(by='f1_results', ascending=False).reset_index(drop=True)

print(f'''Лучшая модель: {results1["models"][0]} при методе "{results1["method"][0]}"
Показала результат f1: {round(results1["f1_results"][0], 3)} и auc-roc: {round( results1["auc_roc_results"][0], 3)} 
\n Таблица результатов:''')
display(results1)

### Downsempling

In [24]:
undersample = RandomUnderSampler(random_state=12345)
features_train_down, target_train_down = undersample.fit_resample(features_train, target_train)

In [25]:
best_model_random = None                   # лучшая модель случайного леса
best_random_forest_f1_score = 0            # лучший результат f1-score для случайного леса
best_auc_roc_score_model_random_forest = 0 # ллучший результат auc-roc для случайного леса



for d in range(1, len(features_train.columns)):
    for n_e in range(10, 200, 10):
        model_random_forest = RandomForestClassifier(max_depth=d,
                                                    n_estimators=n_e,
                                                    random_state=12345)
        model_random_forest.fit(features_train_down, target_train_down)
        predictions = model_random_forest.predict(features_valid)
        result = f1_score(target_valid, predictions)
        result_roc = roc_auc_score(target_valid, model_random_forest.predict_proba(features_valid)[:, 1])
        if result > best_random_forest_f1_score:
            best_random_forest_f1_score = result
            best_auc_roc_score_model_random_forest = result_roc
            best_model_random = model_random_forest
        else:
            continue





# создадим списки с результатми 

models = [best_model_tree, best_model_random]
f1_results = [best_tree_f1_score, best_random_forest_f1_score]
auc_roc_results = [best_auc_roc_score_model_tree, best_auc_roc_score_model_random_forest]
method = (f'{method}/' * len(models)).split('/')[:-1] # технический список. В него будет вносится метод

In [26]:
results2 = pd.DataFrame({
                                    'models': models,
                                    'f1_results': f1_results,
                                    'auc_roc_results': auc_roc_results,
                                    'method': method
                                }).sort_values(by='f1_results', ascending=False).reset_index(drop=True)

print(f'''Лучшая модель: {results2["models"][0]} при методе "{results2["method"][0]}"
Показала результат f1: {round(results2["f1_results"][0], 3)} и auc-roc: {round( results2["auc_roc_results"][0], 3)} 
\n Таблица результатов:''')
display(results2)

### Class-weight-balance

In [30]:
method = 'Class-weight-balance'            # метод, который тестируем
best_model_random = None                   # лучшая модель случайного леса
best_random_forest_f1_score = 0            # лучший результат f1-score для случайного леса
best_auc_roc_score_model_random_forest = 0 # ллучший результат auc-roc для случайного леса




for d in range(1, len(features_train.columns)):
    model_random_forest = RandomForestClassifier(max_depth=d,
                                                    n_estimators=n_e,
                                                    random_state=12345, class_weight='balanced')
    model_random_forest.fit(features_train, target_train)
    predictions = model_random_forest.predict(features_valid)
    result = f1_score(target_valid, predictions)
    result_roc = roc_auc_score(target_valid, model_random_forest.predict_proba(features_valid)[:, 1])
    if result > best_random_forest_f1_score:
        best_random_forest_f1_score = result
        best_auc_roc_score_model_random_forest = result_roc
        best_model_random = model_random_forest
    else:
        continue



# создадим списки с результатми 

models = [best_model_tree, best_model_random]
f1_results = [best_tree_f1_score, best_random_forest_f1_score]
auc_roc_results = [best_auc_roc_score_model_tree, best_auc_roc_score_model_random_forest]
method = (f'{method}/' * len(models)).split('/')[:-1] # технический список. В него будет вносится метод

In [ ]:
results3 = pd.DataFrame({
                                    'models': models,
                                    'f1_results': f1_results,
                                    'auc_roc_results': auc_roc_results,
                                    'method': method
                                }).sort_values(by='f1_results', ascending=False).reset_index(drop=True)

print(f'''Лучшая модель: {results3["models"][0]} при методе "{results3["method"][0]}"
Показала результат f1: {round(results3["f1_results"][0], 3)} и auc-roc: {round( results3["auc_roc_results"][0], 3)} 
\n Таблица результатов:''')
display(results3)

In [ ]:
df=pd.concat([results1, results2, results3]).sort_values(by='f1_results', ascending=False).reset_index(drop=True)
print(f'''Лучшая модель: {df["models"][0]} 

Показала результат f1: {round(df["f1_results"][0], 3)} и auc-roc: {round( df["auc_roc_results"][0], 3)} 

Таблица результатов:''')
display(df)


In [ ]:
model = df["models"][0] # запишем лучшую модель в переменную model и будем использовать её до конца исследования.

**Наиболее точной моделью, показывающей наиболее точный результат оказалась модель случайного леса. В качестве наиболее оптимального метода борьбы с дисбалансом нам подошёл метод взвешивания классов.**

**ROC-AUC кривая**

In [ ]:
model.fit(features_train, target_train)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

fpr, tpr, thresholds = roc_curve(target_valid, probabilities_one_valid)

plt.figure()

plt.plot(fpr, tpr)

plt.plot([0, 1], [0, 1], linestyle='--')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')

print('Метрика roc_auc:', "{:.2f}".format(roc_auc_score(target_valid, (model.predict_proba(features_valid)))))
plt.show()

Из графика выше видно, насколько наша модель точнее, чем случайная.

## Тестирование модели

**На завершающем этапе необходимо протестировать отобранную модель.**

Тестирование будет проходить в 3 этапа:

    1) померяем f1_score на тестовой выборке;
    2) проведём проверку модели на адекватность (сравним с константой);
    3) оценим recall и поясним результаты модели.

In [ ]:
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print('На тестовой выборке модель показала результат f1-меры, соотвествующий значению:', round(f1_score(target_test, predictions), 2), '\n', 
     'это больше, чем результат, который требовался в первоначальной задаче')

In [ ]:
consant_model = [1 for _ in range(len(predictions))]
print('Случайная модель показывает f1-меру:', round(f1_score(target_test, consant_model), 2), '\n',
      'это практически в 2 раза меньше, чем наша модель. Что подтверждает её адекватность')

In [ ]:
print('С вероятностью в', round(recall_score(target_test, predictions), 2), 'модель правильно предскажет, какой клиент уйдёт')

**Общий вывод:**

В ходе исследования была проведена подготовка и предобработка данных, с помощью этих данных было проведено тренировочное тестирование 3 моделей: случайный лес, древо-решений, а также логистическая регрессия. Первое тренировочное тестирование показало, что сделать корректную модель не проработов с дисбалансом классов - в данном случае не получится. Поэтому была проведена аналитика дисбалансов классов. Было использовано 3 подхода:

    1) апсемплинг;
    2) даунсеплинг;
    3) взвешивание классов.
 
В результате проведения тестирований, был отобран наиболее оптимальный подход, а именнно, подход взвешивания классов. 
В результате тестирования моделей с новым подходоим борьбы с дисбалансом, была выбрана лучшая модель. 
Модель - случайного леса, с гиперпараметрами:  (class_weight='balanced', max_depth=10, n_estimators=80).

Эта модель также была обкатана на тестовых данных. В резульатет чего, F1-мера модели составила 0.61, что больше, чем требовалось в первоисходной задаче. Модель была протестирована на случайных данных и также показала свою эффективность.
Была также оценена плотность модели, говорящая о том, какую долю верных предсказаний отразила модель. Эта доля составлиа 0.61, т.е. модель с 61% вероятностью определяет, с каким клиентом нужно поработать маркетолагам из "Бета-Банка".